<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import besca as bc
import scanpy as sc
import pandas as pd
import numpy as np

/usr/local/anaconda3/envs/besca/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex
/usr/local/anaconda3/envs/besca/lib/python3.7/site-packages/scanpy/api/__init__.py:6: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
/usr/local/anaconda3/envs/besca/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



In [2]:
# Set up data loading
sample_strings = ['Duo_M1', 'Duo_M2', 'Jej_M1', 'Jej_M2', 'Il_M1', 'Il_M2']
sample_id_strings = ['3', '4', '5', '6', '7', '8']
file_base = './GSE92332_RAW/GSM283657'
exp_string = '_Regional_'
data_file_end = '_matrix.mtx.gz'
barcode_file_end = '_barcodes.tsv.gz'
gene_file_end = '_genes.tsv.gz'

In [3]:
# First data set load & annotation
#Parse Filenames
sample = sample_strings.pop(0)
sample_id = sample_id_strings.pop(0)
data_file = file_base+sample_id+exp_string+sample+data_file_end
barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
gene_file = file_base+sample_id+exp_string+sample+gene_file_end

#Load data
adata = sc.read(data_file, cache=True)
adata = adata.transpose()
adata.X = adata.X.toarray()

barcodes = pd.read_csv(barcode_file, header=None, sep='\t')
genes = pd.read_csv(gene_file, header=None, sep='\t')

#Annotate data
barcodes.rename(columns={0:'barcode'}, inplace=True)
barcodes.set_index('barcode', inplace=True)
adata.obs = barcodes
adata.obs['sample'] = [sample]*adata.n_obs
adata.obs['region'] = [sample.split("_")[0]]*adata.n_obs
adata.obs['donor'] = [sample.split("_")[1]]*adata.n_obs

genes.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
genes.set_index('gene_symbol', inplace=True)
adata.var = genes

In [4]:
# Loop to load rest of data sets
for i in range(len(sample_strings)):
    #Parse Filenames
    sample = sample_strings[i]
    sample_id = sample_id_strings[i]
    data_file = file_base+sample_id+exp_string+sample+data_file_end
    barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
    gene_file = file_base+sample_id+exp_string+sample+gene_file_end
    
    #Load data
    adata_tmp = sc.read(data_file, cache=True)
    adata_tmp = adata_tmp.transpose()
    adata_tmp.X = adata_tmp.X.toarray()

    barcodes_tmp = pd.read_csv(barcode_file, header=None, sep='\t')
    genes_tmp = pd.read_csv(gene_file, header=None, sep='\t')
    
    #Annotate data
    barcodes_tmp.rename(columns={0:'barcode'}, inplace=True)
    barcodes_tmp.set_index('barcode', inplace=True)
    adata_tmp.obs = barcodes_tmp
    adata_tmp.obs['sample'] = [sample]*adata_tmp.n_obs
    adata_tmp.obs['region'] = [sample.split("_")[0]]*adata_tmp.n_obs
    adata_tmp.obs['donor'] = [sample.split("_")[1]]*adata_tmp.n_obs
    
    genes_tmp.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
    genes_tmp.set_index('gene_symbol', inplace=True)
    adata_tmp.var = genes_tmp
    adata_tmp.var_names_make_unique()

    # Concatenate to main adata object
    adata = adata.concatenate(adata_tmp, batch_key='sample_id')
    #adata.var['gene_id'] = adata.var['gene_id-1'] #does an errormessage not quite clear why?
    #adata.var.drop(columns = ['gene_id-1', 'gene_id-0'], inplace=True) #does an errormessage not quite clear why?
    adata.obs.drop(columns=['sample_id'], inplace=True)
    adata.obs_names = [c.split("-")[0] for c in adata.obs_names]
    adata.obs_names_make_unique(join='_')

Making variable names unique for controlled concatenation.


In [5]:
#reformat to comply with standard besca structure

#Assign variable names and gene id columns
adata.var_names = [g.split("_")[1] for g in adata.var_names]
adata.var['ENSEMBL'] = [g.split("_")[1] for g in adata.var['gene_id']]
adata.var['SYMBOL'] = adata.var.index.tolist()
adata.var.drop(columns = 'gene_id', inplace = True)

In [6]:
obs = pd.DataFrame({'CELL':adata.obs_names, 'CONDITION':'healthy', 'sample_type':'mouse_small_intestine_epithelial'})
obs.index = obs.CELL
obs = pd.concat([obs,adata.obs], axis = 1)
adata.obs = obs

In [7]:
#make barcodes unique to this experiment
adata.obs.CELL = ['haber_intestine_donor_'+x +'_' + y +'.'+z for x, y, z in zip(adata.obs.donor, adata.obs.region, adata.obs.CELL)]
adata.obs.index = adata.obs.CELL.tolist()
adata.obs_names = adata.obs.index.tolist()

In [8]:
# Annotate the data sets
print(adata.obs['region'].value_counts())
print('')
print(adata.obs['donor'].value_counts())
print('')
print(adata.obs['sample'].value_counts())

Duo    4554
Jej    4496
Il     4303
Name: region, dtype: int64

M2    6711
M1    6642
Name: donor, dtype: int64

Jej_M2    2668
Il_M1     2466
Duo_M1    2348
Duo_M2    2206
Il_M2     1837
Jej_M1    1828
Name: sample, dtype: int64


In [134]:
adata.obs.head()

,CELL,CONDITION,sample_type,donor,region,sample
haber_intestine_donor_M1_Duo.AAACATACAGCGGA,haber_intestine_donor_M1_Duo.AAACATACAGCGGA,healthy,mouse_small_intestine_epithelial,M1,Duo,Duo_M1
haber_intestine_donor_M1_Duo.AAACATACCTTACT,haber_intestine_donor_M1_Duo.AAACATACCTTACT,healthy,mouse_small_intestine_epithelial,M1,Duo,Duo_M1
haber_intestine_donor_M1_Duo.AAACATACTTTGCT,haber_intestine_donor_M1_Duo.AAACATACTTTGCT,healthy,mouse_small_intestine_epithelial,M1,Duo,Duo_M1
haber_intestine_donor_M1_Duo.AAACCGTGCAGTCA,haber_intestine_donor_M1_Duo.AAACCGTGCAGTCA,healthy,mouse_small_intestine_epithelial,M1,Duo,Duo_M1
haber_intestine_donor_M1_Duo.AAACGCTGCAGTCA,haber_intestine_donor_M1_Duo.AAACGCTGCAGTCA,healthy,mouse_small_intestine_epithelial,M1,Duo,Duo_M1


In [135]:
#import public dataset annotation
labels = pd.read_csv('./atlas_metadata.txt', sep = '\t').drop(0)

In [136]:
labels

,NAME,Mouse,Cluster,nGene,Cell-cycle,Region,Condition
1,AAACATTGTTTGGG_M1,M1,Enterocyte.Immature.Distal,1829,-0.18015439,NaN,NaN
2,AAACCGTGCCAAGT_M1,M1,Tuft,1305,0.065477717,NaN,NaN
3,AAACGCACAGCCTA_M1,M1,Goblet,2215,-0.292703976,NaN,NaN
4,AAACGCACCAGATC_M1,M1,Enteroendocrine,2789,-0.332670356,NaN,NaN
5,AAACGCACCGCCTT_M1,M1,Stem,2094,-0.168716659,NaN,NaN
...,...,...,...,...,...,...,...
28719,B10_TTTCACGACAAGCT_Salmonella_TA,Salm_2,TA,1234,-0.0668674247083511,NaN,Salmonella
28720,B10_TTTCAGTGAGGCGA_Salmonella_Enterocyte,Salm_2,Enterocyte,2099,0.151407420185812,NaN,Salmonella
28721,B10_TTTCAGTGCGACAT_Salmonella_Stem,Salm_2,Stem,1596,0.0477913721881939,NaN,Salmonella
28722,B10_TTTCAGTGTGACCA_Salmonella_Endocrine,Salm_2,Endocrine,2966,0.188539206418603,NaN,Salmonella


In [137]:
#make annotation congruent with adata
labels = labels.replace({'m1': 'M1', 'm2':'M2', 'Jejunum': 'Jej', 'Ileum':'Il', 'Duodenum':'Duo'})

In [138]:
labels.Region.value_counts()

Jej    3970
Il     3927
Duo    3768
Name: Region, dtype: int64

In [139]:
#extract cells contained within this dataset
labels_regions = labels[[x in ['Jej', 'Il','Duo'] for x in labels.Region.tolist()]]

#reformat barcodes to match unique ones generated above
labels_regions['CELL'] = ['haber_intestine_donor_'+ mouse + '_' +region+ '.' + x.split('_')[0] for mouse, region, x in zip(labels_regions.Mouse.tolist(), labels_regions.Region.tolist(), labels_regions.NAME.tolist())]
labels_regions.index = labels_regions.CELL

/usr/local/anaconda3/envs/besca/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [140]:
#get annotation
annotation = labels_regions.Cluster.to_frame()
annotation.columns = ['celltype_haber']

#extract barcodes of interest
merged = pd.merge(adata.obs, annotation, left_index = True, right_index = True)

#annotate cells
adata.obs['celltype_haber'] = [merged.loc[x, 'celltype_haber'] if x in merged.CELL.tolist() else'not determined'for x in adata.obs.CELL.tolist()]

In [141]:
print(adata.obs.celltype_haber.value_counts())
print('')
print(merged.region.value_counts())

not determined     9469
Stem               1143
Goblet              898
EP                  640
Enterocyte          563
TA                  266
Paneth              203
Enteroendocrine      92
Tuft                 79
Name: celltype_haber, dtype: int64

Il     3873
Duo      11
Name: region, dtype: int64


In [142]:
bc.export.X_to_mtx(adata=adata, outpath = './raw', write_metadata = True )

writing out matrix.mtx ...
adata.X successfully written to matrix.mtx. 
genes successfully written out to genes.tsv
cellbarcodes successfully written out to barcodes.tsv
annotation successfully written out to metadata.tsv
